# Note

> Basic atomic building block for music

In [ ]:
#|default_exp note

In [ ]:
#|hide
from nbdev.showdoc import *

In [ ]:
#|export
import io
import numpy as np
from fastcore.all import *
import scipy.io.wavfile as wav
from IPython.display import Audio
from mingus.core import notes as mingus_notes

In [ ]:
#|export
BASE_NOTES = ["C", "D", "E", "F", "G", "A", "B"]
CHROMATIC_NOTES = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
INTERVALS = ["1", "b2", "2", "b3", "3", "4", "#4", "5", "b6", "6", "b7", "7"]
NOTE_MAPPING = {
    "C": 0,
    "C#": 1,
    "Db": 1,
    "D": 2,
    "D#": 3,
    "Eb": 3,
    "E": 4,
    "Fb": 4,
    "E#": 5,
    "F": 5,
    "F#": 6,
    "Gb": 6,
    "G": 7,
    "G#": 8,
    "Ab": 8,
    "A": 9,
    "A#": 10,
    "Bb": 10,
    "B": 11,
    "Cb": 11,
    "B#": 0
}
STEPS_TO_INTERVAL = {
    0: "1",
    1: "b2",
    2: "2",
    3: "b3",
    4: "3",
    5: "4",
    6: "b5",
    7: "5",
    8: "b6",
    9: "6",
    10: "b7",
    11: "7",
}

STEPS_TO_INTERVAL_FULL = {
    0: "unison",
    1: "minor second",
    2: "major second",
    3: "minor third",
    4: "major third",
    5: "perfect fourth",
    6: "tritone",
    7: "perfect fifth",
    8: "minor sixth",
    9: "major sixth",
    10: "minor seventh",
    11: "major seventh",
}
INTERVAL_TYPES = {
    0: "Perfect Consonant", # Unison
    1: "Sharp Dissonant", # Minor Second
    2: "Mild Dissonant", # Major Second
    3: "Soft Consonant", # Minor Third
    4: "Soft Consonant", # Major Third
    5: "Contextual", # Perfect Fourth
    6: "Neutral", # Tritone/Augmented Fourth/Diminished Fifth
    7: "Perfect Consonant", # Perfect Fifth
    8: "Soft Consonant", # Minor Sixth
    9: "Soft Consonant", # Major Sixth
    10: "Mild Dissonant", # Minor Seventh
    11: "Sharp Dissonant", # Major Seventh
}

In [ ]:
#|hide
for item in [BASE_NOTES, CHROMATIC_NOTES, INTERVALS]:
    assert len(item) == len(set(item)), f"Duplicate {item} found in {item}"
assert len(CHROMATIC_NOTES) == 12
assert len(INTERVALS) == 12

The `Note` is the basic atomic unit in music.

In [ ]:
#|export
class Note(BasicRepr):
    def __init__(self, note: str, oct: int = 4):
        assert isinstance(oct, int) and oct > 0, f"Octave must be a positive integer, got oct={oct}."
        store_attr()
        # Transform note to uppercase
        if isinstance(note, str):
            note = note[0].upper() + note[1:]
            assert mingus_notes.is_valid_note(note), f"Note '{note}' is not valid"
            self.note = self.postprocess_note(mingus_notes.remove_redundant_accidentals(note))

    @staticmethod
    def postprocess_note(note: str):
        """ Get rid of unnecessary accidentals."""
        if note == "B#": note = "C"
        elif note == "E#": note = "F"
        elif note == "Cb": note = "B"
        elif note == "Fb": note = "E"
        elif note.endswith("##"):
            note = BASE_NOTES[BASE_NOTES.index(note[0])+1]
        elif note.endswith("bb"):
            note = BASE_NOTES[BASE_NOTES.index(note[0])-1]
        return str(note)
    
    def __str__(self): return self.note
    def __int__(self): return NOTE_MAPPING[str(self)]
    def rel(self): return self.oct * 12 + int(self)
    def __eq__(self, other): return self.rel() == other.rel()
    def __ne__(self, other): return self.rel() != other.rel()
    def __lt__(self, other): return self.rel() < other.rel()
    def __le__(self, other): return self.rel() <= other.rel()
    def __gt__(self, other): return self.rel() > other.rel()
    def __ge__(self, other): return self.rel() >= other.rel()

## Initialization

Every note has an accompanying octave `oct` associated with it. This is useful later for determining intervals and playing the sound of the notes.

In [ ]:
a_sharp = Note("A#")
a_sharp

Note(note='A#', oct=4)

## Integer Representation

Each note has an integer value which shows its place in a C octave (`C == 0`, `C# == 1`, `B == 11`, etc.)

In this example `A#` should denote `10` as its integer value.

In [ ]:
assert int(a_sharp) == 10
int(a_sharp)

10

## String Representation

In [ ]:
assert str(a_sharp) == "A#"
a_sharp.note

'A#'

In [ ]:
c_sharp = Note("C#")
c_sharp

Note(note='C#', oct=4)

## Comparison

Notes can be compared with each other using familiar Python operators. Here are some examples of comparison between `Note` objects.

In [ ]:
assert Note("E#") == Note("F")
assert Note("B#") == Note("C")
assert Note("C##") == Note("D")
assert Note("Fb") == Note("E")
assert Note("Abb") == Note("G")
assert Note("Bbb") == Note("A")
assert Note("Cb") == Note("B")
assert Note("C") == Note("C")
assert Note("E#") == Note("F")
assert Note("A#") != Note("B")
assert Note("F") > Note("C#")
assert Note("B#") <= Note("C")
assert Note("C") < Note("B")
assert Note("E") >= Note("E")
assert Note("C", oct=4) > Note("B", oct=3)
assert Note("C", oct=4) != Note("C", oct=5)
assert Note("C", oct=4) < Note("C", oct=5)

## Adding

### Semitones

Adding semitones to a note will return a new note with `n` semitones added above the original note.

For example, adding 1 semitone to `A#` (A Sharp) will return `B`.

Adding `Note` objects together will form a `Chord`. More on that in the `Chord` section.

In [ ]:
#|export
@patch
def __add__(self:Note, other):
    """Add n semitones to a note."""
    octave_change = (other + int(self)) // 12
    return Note(CHROMATIC_NOTES[(int(self) + other) % 12], oct=self.oct + octave_change)

In [ ]:
for i in range(1, 13):
    print(a_sharp+i)

B
C
C#
D
D#
E
F
F#
G
G#
A
A#


The octave of the note is automatically updated when adding.

In [ ]:
assert (a_sharp+1).oct == a_sharp.oct
assert (a_sharp+2).oct == a_sharp.oct + 1
assert (Note("C")+11).oct == Note("C").oct

In [ ]:
assert Note("C")+11 == Note("B")
assert str(a_sharp+1) == "B"
assert str(a_sharp+11) == "A"
assert str(a_sharp+12) == "A#"
assert str(a_sharp+13) == "B"
assert str(a_sharp+47) == "A"
assert str(a_sharp+0) == "A#"

### Whole Notes

The `%` operator is a shortcut for adding whole notes.

In [ ]:
#|export
@patch
def __mod__(self:Note, other):
    """Add n whole notes."""
    return self + other * 2

In [ ]:
assert a_sharp % 1 == Note("C", oct=5)
a_sharp % 1

Note(note='C', oct=5)

In [ ]:
for i in range(1, 7):
    print(a_sharp % i)

C
D
E
F#
G#
A#


In [ ]:
assert str(a_sharp % 1) == "C"
assert str(a_sharp % 0) == "A#"
assert str(a_sharp % 6) == "A#"
assert str(a_sharp % 6 + 1) == "B"

## Subtraction

### Semitones


Subtracting semitones from a `Note` returns a new note with `n` semitones subtracted from the original note.

For example, subtracting 1 semitone from `C` returns `B`. Subtracting 1 semitone from `A#` returns `A`.


In [ ]:
#|export
@patch
def __sub__(self:Note, other):
    """Subtract n semitones from a note."""
    octave_change = (other + int(self)) // 12
    return Note(CHROMATIC_NOTES[(int(self) - other) % 12], oct=self.oct - octave_change)

In [ ]:
for i in range(1, 13):
    print(a_sharp-i)

A
G#
G
F#
F
E
D#
D
C#
C
B
A#


In [ ]:
assert (a_sharp-1).oct == a_sharp.oct
assert (a_sharp-10).oct == a_sharp.oct - 1
assert (a_sharp-6).oct == a_sharp.oct - 1
assert (a_sharp-12).oct == a_sharp.oct - 1

In [ ]:
assert str(a_sharp-0) == "A#"
assert str(a_sharp-1) == "A"
assert str(a_sharp-11) == "B"
assert str(a_sharp-12) == "A#"
assert str(a_sharp-13) == "A"
assert str(a_sharp-25) == "A"

### Whole Notes

The `//` operator is a shortcut for subtracting whole notes.

In [ ]:
#|export
@patch
def __floordiv__(self:Note, other):
    """Subtract n whole notes"""
    return self - other * 2

In [ ]:
assert a_sharp // 1 == Note("G#", oct=3)
a_sharp // 1

Note(note='G#', oct=3)

In [ ]:
for i in range(1, 7):
    print(a_sharp // i)

G#
F#
E
D
C
A#


In [ ]:
assert str(a_sharp // 1) == "G#"
assert str(a_sharp // 0) == "A#"
assert str(a_sharp // 6) == "A#"
assert str(a_sharp // 6 + 1) == "B"

## Convert to Major or Minor

`Note` objects can be converted to its relative major or minor. How this is converted is visualized on the circle of fifths.

<img src="https://upload.wikimedia.org/wikipedia/commons/3/33/Circle_of_fifths_deluxe_4.svg" width="40%" alt="Circle of Fifths">

For example, the relative minor of C is A. The relative major of C# is E.

`minor` converts an arbitrary note to its relative minor. This means 3 semitones are subtracted from the note.

In [ ]:
#|export
@patch
def minor(self:Note): return self - 3

In [ ]:
c = Note("C")
assert str(c.minor()) == "A"
c.minor()

Note(note='A', oct=4)

`major` converts an arbitrary note to its relative major. This means 3 semitones are added to the note.

In [ ]:
#|export
@patch
def major(self:Note): return self + 3

In [ ]:
assert str(c_sharp.major()) == "E"
c_sharp.major()

Note(note='E', oct=4)

## Play Audio

Every `Note` can be played as audio. A `Note` is first transformed into a bytestring.

In [ ]:
#|export
@patch 
def get_audio_array(self:Note, length=1, sr=44100):
    a = {'C':0,'C#':1,'Db':1,'D':2,'D#':3,'Eb':3,'E':4,'F':5,
         'F#':6,'Gb':6,'G':7,'G#':8,'Ab':8,'A':9,'A#':10,'Bb':10,'B':11}
    t = np.linspace(0, length, int(sr * length), False)
    def freq(n): return 440 * 2**((12 * (int(n[-1])+1) + a[n[:-1]] - 69)/12)
    wave = np.sin(2 * np.pi * freq(f"{self.note}{self.oct}") * t)
    wave = (wave / np.max(np.abs(wave)) * 32767).astype(np.int16)
    return wave

@patch
def get_audio_bytes(self:Note, length=1, sr=44100):
    buf = io.BytesIO(); wav.write(buf, sr, self.get_audio_array(length, sr))
    return buf.getvalue()

In [ ]:
c.get_audio_bytes(length=5)[:50]

b'RIFF\xcc\xba\x06\x00WAVEfmt \x10\x00\x00\x00\x01\x00\x01\x00D\xac\x00\x00\x88X\x01\x00\x02\x00\x10\x00data\xa8\xba\x06\x00\x00\x00\xc5\x04\x88\t'

For convenience, we can also play the audio directly.

In [ ]:
#|export
@patch
def play(self:Note, length=1): return Audio(data=self.get_audio_bytes(length))

In [ ]:
c.play()

THe length of a specific note can be specified for the audio.

In [ ]:
c.play(length=2)

In [ ]:
a_sharp.play()

Also here, method chaining is possible.

In [ ]:
(a_sharp - 6).play()

# Interval

Two `Note` objects can be combined to form an `Interval`.

In [ ]:
#|export 
class Interval:
    def __init__(self, note1: Note, note2: Note):
        store_attr()
        self.notes = [note1, note2]
        self.semitones = self.set_semitones()
    
    @property
    def short(self): return STEPS_TO_INTERVAL[abs(self.semitones)]
    @property
    def long(self): return STEPS_TO_INTERVAL_FULL[abs(self.semitones)]

    def set_semitones(self):
        # TODO Add upper extensions to semitone calculation and interval names (2+ octaves)
        return (12 + int(self.note2) - int(self.note1) + (self.note2.oct - self.note1.oct) * 12) % 12

    def __repr__(self): return f"{str(self.long)} ({str(self.short)})"
    def __eq__(self, other): return self.semitones == other.semitones
    def __ne__(self, other): return not self.semitones == other.semitones
    def __lt__(self, other): return self.semitones < other.semitones
    def __le__(self, other): return self.semitones <= other.semitones
    def __gt__(self, other): return self.semitones > other.semitones
    def __ge__(self, other): return self.semitones >= other.semitones
    def __abs__(self): 
        c = Interval(self.note1, self.note2)
        c.semitones = abs(c.semitones)
        return c

## Initialization

Calling `interval` on a `Note` object requires providing another `Note` object and returns an `Interval` object. A shortcut is to use the `&` operator and has the same effect.

In [ ]:
#|export
@patch
def interval(self:Note, other:Note): return Interval(self, other)

@patch
def __and__(self:Note, other:Note): return self.interval(other)

## Comparison

The full name of the interval between `A#` and `C#` is a `minor third`.

In [ ]:
c_sharp = Note("C#")
m3 = a_sharp.interval(c_sharp)
m3

minor third (b3)

From here on in these docs we'll use the more compact `&` syntax.

In [ ]:
assert m3 == a_sharp & c_sharp
a_sharp & c_sharp

minor third (b3)

In [ ]:
assert m3.semitones == 3
m3.semitones

3

The shorthand for a `minor third` interval is `b3`.

In [ ]:
assert m3.long == "minor third"
m3.long

'minor third'

In [ ]:
assert m3.short == "b3"
m3.short

'b3'

In [ ]:
ninth = Note("G#", oct=5) & Note("A", oct=6)
assert ninth.semitones == 1
ninth

minor second (b2)

`Interval` objects can be compared with each other. For a single octave the minor 3rd as larger than the ninth (i.e. minor 2nd).

In [ ]:
assert m3 >= ninth
assert m3 != ninth
assert not m3 < ninth

Note that with negative intervals its a different story. `C#` compared with `A#` is a major 6th.

In [ ]:
low_a_sharp = Note("A#", oct=3)
neg_m3 = c_sharp & low_a_sharp
neg_m3

major sixth (6)

In [ ]:
assert neg_m3 > m3
assert neg_m3 != m3

In [ ]:
#|hide
assert (Note("C", oct=5) & Note("D", oct=5)).semitones == 2
assert (Note("C", oct=2) & Note("E", oct=2)).short == "3"
assert (Note("C", oct=5) & Note("A", oct=5)).long == "major sixth"
assert (Note("C", oct=5) & Note("B", oct=5)).long == "major seventh"
assert (Note("C", oct=5) & Note("D", oct=5)).long == "major second"
assert (Note("C", oct=5) & Note("C", oct=6)).long == "unison"
assert (Note("C", oct=5) & Note("C", oct=7)).long == "unison"
assert (Note("C", oct=5) & Note("D", oct=6)).notes == [Note("C", oct=5), Note("D", oct=6)]
assert (Note("C", oct=5) & Note("D", oct=5)).notes == [Note("C", oct=5), Note("D", oct=5)]

## Interval Type

An `Interval` can also determine its type. An interval can be:

- `Perfect consonant` (Unison, Octave and 5th)
- `Soft consonant` (3rds and 6ths), 
- `Mild Dissonant` (Minor 7th and Major 2nd), 
- `Sharp Dissonant` (Major 7th and Minor 2nd), 
- `Contextual` (4th)
- `Neutral` (Tritone).

In [ ]:
#|export
@patch
def type(self:Interval): return INTERVAL_TYPES[abs(self.semitones) % 12]

A fifth is a `Perfect Consonant`.

In [ ]:
fifth = Note("C") & Note("G")
assert fifth.type() == "Perfect Consonant"
fifth.type()

'Perfect Consonant'

A minor second is `Sharp Dissonant`.

In [ ]:
m2 = Note("C") & Note("C#")
assert m2.type() == "Sharp Dissonant"
m2.type()

'Sharp Dissonant'

A special case is the fourth which is `Contextual`. This can be soft consonant or dissonant depending on the context of harmonic movement and is not feasible to determine within one interval.

In [ ]:
fourth = Note("C") & Note("F")
assert fourth.type() == "Contextual"
fourth.type()

'Contextual'

The `Interval.type` method handles upper extensions by comparing it within the same octave. For example a flat 9 (`b9`) is treated as a minor second and therefore `Sharp Dissonant`.

In [ ]:
ninth = Note("B", oct=4) & Note("C", oct=5)
ninth.type()

'Sharp Dissonant'

## Adding and subtracting

You can add to `Interval` objects to augment or diminish them.

In [ ]:
#|export
@patch
def __add__(self:Interval, other):
    return Interval(self.note1, self.note2+other)

@patch 
def __sub__(self:Interval, other):
    return Interval(self.note1, self.note2-other)

In this example, we augment a minor 3rd interval to a major 3rd (`b3`->`3`)

In [ ]:
m3 = a_sharp & c_sharp
m3

minor third (b3)

In [ ]:
assert (m3 + 1).long == "major third"
m3 + 1

major third (3)

If we diminish the minor 3rd interval it becomes a major 2nd (`b3`->`2`).

In [ ]:
assert (m3 - 1).long == "major second"
m3 - 1

major second (2)

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()